# Colab setup

In [1]:
import sys
if 'google.colab' in str(get_ipython()):
    ! {sys.executable} -m pip install pytorch-lifestream

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

# import logging
import torch
import pytorch_lightning as pl
# import warnings

# warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

In [3]:
%load_ext pyinstrument

## Data preproccessing

In [4]:
import os
import pandas as pd

source_data = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true', compression='gzip')
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [5]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
    return_records=True,
)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
%%pyinstrument

dataset = preprocessor.fit_transform(source_data)

_     ._   __/__   _ _  _  _ _/_   Recorded: 13:03:59  Samples:  17403
 /_//_/// /_\ / //_// / //_'/ //     Duration: 45.879    CPU time: 45.297
/   _/                      v4.6.2

Program: c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py -f C:\Users\user\AppData\Roaming\jupyter\runtime\kernel-1d3925b1-2dab-4fb8-a868-1f1a994bedb7.json

45.879 <module>  ..\..\..\Temp\ipykernel_13500\382335421.py:2
`- 45.879 PandasDataPreprocessor.wrapped  sklearn\utils\_set_output.py:138
   `- 45.879 PandasDataPreprocessor.fit_transform  ptls\preprocessing\base\data_preprocessor.py:41
      `- 45.649 UserGroupTransformer.wrapped  sklearn\utils\_set_output.py:138
            [3 frames hidden]  sklearn
               45.649 UserGroupTransformer.fit_transform  sklearn\base.py:850
               |- 37.384 UserGroupTransformer.wrapped  sklearn\utils\_set_output.py:138
               |  |- 27.336 UserGroupTransformer.transform  ptls\preprocessing\pandas\user_group_transformer.py:63
               |  |  |- 18.883 DataFrameGroupBy.apply  pandas\core\groupby\groupby.py:1725
               |  |  |     [15 frames hidden]  pandas
               |  |  |        18.289 BaseGrouper.apply_groupwise  pandas\core\groupby\ops.py:883
               |  |  |        |- 14.340 UserGroupTransformer.df_to_feature_arrays  ptls\preprocessing\pandas\user_group_transformer.py:49
               |  |  |        |  |- 7.434 Series.__init__  pandas\core\series.py:371
               |  |  |        |  |     [13 frames hidden]  pandas, torch, <built-in>
               |  |  |        |  |- 3.429 <dictcomp>  ptls\preprocessing\pandas\user_group_transformer.py:60
               |  |  |        |  |  `- 3.358 decide  ptls\preprocessing\pandas\user_group_transformer.py:50
               |  |  |        |  |     `- 1.996 _iLocIndexer.__getitem__  pandas\core\indexing.py:1139
               |  |  |        |  |           [3 frames hidden]  pandas
               |  |  |        |  `- 3.385 DataFrame.to_dict  pandas\core\frame.py:1932
               |  |  |        |        [10 frames hidden]  pandas
               |  |  |- 5.306 DataFrame.sort_index  pandas\core\frame.py:7040
               |  |  |     [9 frames hidden]  pandas, <built-in>
               |  |  |- 1.799 DataFrame.set_index  pandas\core\frame.py:5744
               |  |  |     [11 frames hidden]  pandas
               |  |  `- 0.573 attach_column  ptls\preprocessing\pandas\col_transformer.py:15
               |  |- 8.255 FrequencyEncoder.transform  ptls\preprocessing\pandas\frequency_encoder.py:64
               |  |  |- 5.898 Series.astype  pandas\core\generic.py:6368
               |  |  |     [8 frames hidden]  pandas, <built-in>
               |  |  |- 1.389 Series.map  pandas\core\series.py:4460
               |  |  |     [6 frames hidden]  pandas, <built-in>
               |  |  `- 0.856 attach_column  ptls\preprocessing\pandas\col_transformer.py:15
               |  |     `- 0.590 concat  pandas\core\reshape\concat.py:157
               |  |           [5 frames hidden]  pandas
               |  `- 1.440 ColIdentityEncoder.transform  ptls\preprocessing\pandas\col_identity_transformer.py:21
               |     `- 1.343 attach_column  ptls\preprocessing\pandas\col_transformer.py:15
               |        `- 0.833 concat  pandas\core\reshape\concat.py:157
               |              [5 frames hidden]  pandas
               `- 7.951 FrequencyEncoder.fit  ptls\preprocessing\pandas\frequency_encoder.py:52
                  |- 6.474 Series.astype  pandas\core\generic.py:6368
                  |     [8 frames hidden]  pandas, <built-in>
                  `- 1.477 Series.value_counts  pandas\core\base.py:923
                        [3 frames hidden]  pandas

In [7]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [8]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

len(train), len(test)

(24000, 6000)

In [10]:
train[0].keys()

dict_keys(['client_id', 'trans_date', 'event_time', 'small_group', 'amount_rur'])

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

### Model definition

In [11]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

### Data loader

In [12]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200,
        ),
    ),
    train_num_workers=16,
    train_batch_size=256,
)

### Trainer

In [13]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=15,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else "auto",
    enable_progress_bar=False,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### Training 

In [ ]:
%%pyinstrument
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

Missing logger folder: D:\WORK\Repo\pytorch-lifestream\demo\lightning_logs
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name               | Type            | Params | Mode 
---------------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0      | train
1 | _seq_encoder       | RnnSeqEncoder   | 240 K  | train
2 | _validation_metric | BatchRecallTopK | 0      | train
3 | _head              | Head            | 0      | train
---------------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.962     Total estimated model params size (MB)


logger.version = 0


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\torch\utils\data\dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


### Save sequence encoder for other experiments

In [14]:
torch.save(seq_encoder.state_dict(), "coles-emb.pt")

## Inference 

In [15]:
# embedding inference
%%pyinstrument
from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

train_embeds.shape, test_embeds.shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/kireev/pipenv_envs/pytorch-lifestream-KxQJF1XF/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kireev/pipenv_envs/pytorch-lifestream-KxQJF1XF/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:175: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


(torch.Size([24000, 256]), torch.Size([6000, 256]))

In [16]:
# join target and embeddings

df_target = pd.read_csv('https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true')
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


Obtained embeddings can be used as features for model training

For example:

In [17]:
from sklearn.ensemble import RandomForestClassifier

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5756666666666667